# TCL tutorial


In [1]:
%load_ext tclmagic

#  If the prev line doesnt work  install it using the next line in your unix terminal
#  /depot/Python/Python-3.8.0/bin/pip install -U tcl-magic
#  You can configure it to  

In [5]:
%%tcl 



set script_name [file tail [file rootname [file normalize [info script]]]]
puts "script name --> $script_name"
puts [info script]

script name --> 



In [9]:
%%tcl

proc logMsg {fp msg} {
    puts $fp $msg
}

logMsg stderr 'Message'


'Message'


In [10]:
%%tcl

proc exam {args} {
    set input "This is my world" 
    regexp {(This) (is) (my) (world)} $input all a b c d 
    return [dict create hola $a b $b c $c d $d]
}

set result [exam ...]
dict with result {}
puts $hola
puts $b



This
is


In [11]:
%%tcl

proc run_system_cmd {{command "NaN"} {verbosity 0}} {
    set results ""
    if {$command == "NaN"} { 
        return [lappend results NULL_VAL NULL_VAL NONE]
    }
    
    try {
        set status [exec >@stdout 2>@stderr "$command"]
        lappend results $stdout $stderr $status
    } trap NONE errOut {
        puts $options
    } trap CHILDKILLED {- opts} {
        lassign [dict get $opts -errorcode] -> pid sigName msg
        # process $pid was killed by signal $sigName; message is $msg
        lappend results [list $pid $sigName $msg]
    } trap CHILDSTATUS {- opts} {
        lassign [dict get $opts -errorcode] -> pid code
        # process $pid exited with non-zero exit code $code
        lappend results [list $pid $code]
    } trap CHILDSUSP {- opts} {
        lassign [dict get $opts -errorcode] -> pid sigName msg
        # process $pid was suspended by signal $sigName; message is $msg
        lappend results [list $pid $sigName $msg]
    } trap POSIX {- opts} {
        lassign [dict get $opts -errorcode] -> errName msg
        # Some kind of kernel failure; details in $errName and $msg
        lappend results [list $pid $sigName $msg]
    }
   
    return $results
}

In [12]:
%%tcl

proc tellme {id a e op} {
    puts "  $id a=$a e=$e op=$op\
        ax=[info exists ::$a] ex=[info exists ::${a}($e)]"
}

proc do args {
    puts <$args>
    uplevel 1 $args
}

trace add variable a {write unset} {tellme array}
trace add variable a(1) {write unset} {tellme element}

puts [trace info variable a]
puts [trace info variable a(1)]

do set a(0) zero
do set a(1) one
do set a(2) two
do unset a(0)
do unset a(2)
do unset a


{{write unset} {tellme array}}
{{write unset} {tellme element}}
<set a(0) zero>
  array a=a e=0 op=write ax=1 ex=1
<set a(1) one>
  array a=a e=1 op=write ax=1 ex=1
  element a=a e=1 op=write ax=1 ex=1
<set a(2) two>
  array a=a e=2 op=write ax=1 ex=1
<unset a(0)>
  array a=a e=0 op=unset ax=1 ex=0
<unset a(2)>
  array a=a e=2 op=unset ax=1 ex=0
<unset a>
  array a=a e= op=unset ax=0 ex=0
  element a=a e=1 op=unset ax=0 ex=0


In [8]:
%%tcl

proc who_called {} {
    puts [list {I was called from} [
        lindex [info level -1] 0] {but first call was to} [
        lindex [info level 1] 0]]
}

proc a {} who_called
proc b {} who_called
proc c {} who_called
proc d {} a
proc e n {$n}

In [9]:
%%tcl

proc callstack {} {

    set line_of_procs ""
    for {set i [expr [info level]-1]} {$i > 0} {incr i -1} {

        set level [info level -$i]
        set frame [info frame -$i]

        if {[dict exists $frame proc]} {
            set pname [dict get $frame proc]
            set pargs [lrange $level 1 end]

            if {$line_of_procs == ""} {
                append line_of_procs "$pname "
            } else {
                append line_of_procs " --> $pname "
            }
                        
            foreach arg $pargs {
                append line_of_procs "   * $arg"
            }


        } else {
            append line_of_procs " - **unknown stack item**: $level $frame"
        }
    }

    return [join $line_of_procs]
}

proc Main {} {
    A
}

proc A {} {
    run_system_cmd
}
proc run_system_cmd {} {
     puts [callstack]
  
}


Main

::Main --> ::A --> ::run_system_cmd


In [10]:
%%tcl

proc Main {} {
    A
}

proc A {} {
    run_system_cmd
}

proc run_system_cmd {} {
    puts [callstack]
    try {
        puts [dict get [info frame -1] proc]
    } on error {result options} {
        puts fail-1
    }
    try {
        puts [dict get [info frame -2] proc]
    } on error {result options} {
        puts fail-2
    }
    try {
        puts [dict get [info frame -3] proc]
    } on error {result options} {
        puts fail-3
        puts $result
        puts 
    }
}

Main


::Main --> ::A --> ::run_system_cmd
::A
::Main
fail-3
key "proc" not known in dictionary


TclError: wrong # args: should be "puts ?-nonewline? ?channelId? string"

In [11]:
%%tcl

try {
    set f [open /some/file/name]
} on error {result options} {
    puts {something broke!}
    puts $options
}

something broke!
-code 1 -level 0 -errorstack {INNER {invokeStk1 open /some/file/name}} -errorcode {POSIX ENOENT {no such file or directory}} -errorinfo {couldn't open "/some/file/name": no such file or directory
    while executing
"open /some/file/name"
    ("try" body line 2)} -errorline 2


In [12]:
%%tcl

set longline [string repeat - 20]
append title "$longline TITLE $longline"
puts $title


-------------------- TITLE --------------------


In [13]:
%%tcl

proc eprint {message} {
    set msg "-E- $message"
    print_in_color $msg "red"
}

proc print_in_color {message color} {
    switch $color {
        "red"    {set color_code "1;31m"}
        "green"  {set color_code "1;32m"}
        "yellow" {set color_code "1;33m"}
        "blue"   {set color_code "1;34m"}
        "cyan"  {set color_code "1;36m"}
        "white_on_red" {set color_code "37;41m"}
        default  {set color_code "1m"}
    }
    puts "\033\[$color_code $message \033\[0m"
}


In [14]:
%%tcl

set START_TIME [clock seconds]
read_file tct
after 1000
set END_TIME [clock seconds]
set runtime [expr $START_TIME - $END_TIME]
eprint $START_TIME
eprint $END_TIME
eprint $runtime

TclError: invalid command name "read_file"

In [15]:
%%tcl

set START_TIME [clock seconds]
read_file tct
after 1000
set END_TIME [clock seconds]
set runtime [expr $START_TIME - $END_TIME]
eprint $START_TIME
eprint $END_TIME
eprint $runtime

TclError: invalid command name "read_file"

In [16]:
callable 'hello'

SyntaxError: invalid syntax (2111261449.py, line 1)

In [ ]:
callable

In [ ]:
%%perl

se strict;
use warnings;
use FindBin qw($RealBin $RealScript);

In [ ]:
%lsmagic


In [ ]:
%automagic


In [ ]:
pwd


In [ ]:
%automagic

In [ ]:
%pwd

In [ ]:
callable

<function callable(obj, /)>

In [ ]:
%%perl

se strict;
use warnings;
use FindBin qw($RealBin $RealScript);

Can't locate object method "se" via package "strict" at - line 2.


CalledProcessError: Command 'b'\nse strict;\nuse warnings;\nuse FindBin qw($RealBin $RealScript);\n'' returned non-zero exit status 9.

In [ ]:
%lsmagic


Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cd  %clear  %cls  %colors  %conda  %config  %connect_info  %copy  %ddir  %debug  %dhist  %dirs  %doctest_mode  %echo  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %macro  %magic  %matplotlib  %mkdir  %more  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %ren  %rep  %rerun  %reset  %reset_selective  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %tcl  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%cmd  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%p

In [ ]:
%automagic



Automagic is OFF, % prefix IS needed for line magics.


In [ ]:
pwd


NameError: name 'pwd' is not defined

In [ ]:
%%tcl

set verbosity verbosityaaaa
eprint $verbosity
set v "vaaaa"
eprint $v
eprint [info exists verbosity]
set VERBOSITY [if {[info exists verbosity]} {list $verbosity} ]
eprint $VERBOSITY

 -E- verbosityaaaa 
 -E- vaaaa 
 -E- 1 
 -E- verbosityaaaa 


In [ ]:
%%tcl


proc get_min_val {{value_A ""} {value_B ""}} {
    set A_is_empty [expr {$value_A==""}]
    eprint $A_is_empty
    set B_is_empty [expr {$value_B==""}]
    eprint $B_is_empty
    set both_are_empty [expr {$A_is_empty+$B_is_empty==2}]
    eprint $both_are_empty 
    try {
        if {$A_is_empty} { wprint "The first value is not defined"}
        if {$B_is_empty} { 
            wprint "The second value is not defined"
            if {$A_is_empty} { 
                eprint "There are no values defined."
                return ""
            }
        }
        
        set title [get_caller_proc]
        set init_msg "Ending Function: "
        iprint "$longline $init_msg $title $longline"
    } on error {result options} {
        eprint $result
    }
}

get_min_val 
[string is integer -strict "2"]

 -E- 1 
 -E- 1 
 -E- 1 
 -E- invalid command name "wprint" 


TclError: invalid command name "1"

In [ ]:
%%tcl

proc isa_integer {foo} {
    if {[string is integer -strict $foo]} {
        eprint "foo is an integer!"
        return true
    }
    return false
}

proc get_min_val {{value_A ""} {value_B ""}} {
    set A_is_empty [expr {$value_A==""}]
    eprint $A_is_empty
    set B_is_empty [expr {$value_B==""}]
    eprint $B_is_empty
    set both_are_empty [expr {$A_is_empty+$B_is_empty==2}]
    eprint $both_are_empty
    set A_is_integer [if {[isa_integer $value_A]} {list 1} else {list 0}]
    eprint $A_is_integer
    set B_is_integer [if {[isa_integer $value_B]} {list 1} else {list 0}]
    eprint $B_is_integer
    set both_are_int [expr {$A_is_integer+$B_is_integer==2}]
    eprint $both_are_int
    try {
        if {$both_are_empty} { 
            eprint "Both values are not defined"
            return ""
        } elseif {$A_is_empty} { 
            eprint "The first value is not defined"
            if {$B_is_integer} {
                return $value_B
            } else {
                eprint "The second value is not an integer."
            }
        } elseif {$B_is_empty} { 
            eprint "The second value is not defined"
            if {$A_is_integer} {
                return $value_A
            } else {
                eprint "The first value is not an integer."
            }
        }
        if {$both_are_int} { 
            if {$value_A > $value_B} {
                return $value_B
            } else {
                return $value_A
            }
        } elseif {$A_is_integer} { 
            eprint "The second value is not an integer"
            return $value_A
        } elseif {$B_is_integer} { 
            eprint "The first value is not an integer"
            return $value_B
        } else {
            eprint  "Something is broken."
        }

    } on error {result options} {
        eprint $result
    }
}

eprint [get_min_val 110 12]


 -E- 0 
 -E- 0 
 -E- 0 
 -E- foo is an integer! 
 -E- 1 
 -E- foo is an integer! 
 -E- 1 
 -E- 1 
 -E- 12 


In [ ]:
%%tcl
package require tcltest 2.0

test intro-1 {This is as simple as it gets.} {
        expr 3 * 5
    } 15

TclError: invalid command name "test"

In [ ]:
%%tcl

array set balloon {color red}

array exists balloon1

if {[array exists $balloon]} {eprint $balloon(color)}
if {[array exists balloon1]} {eprint "no deberiamos estar aca"}

TclError: can't read "balloon": variable is array

In [ ]:
%%tcl
proc run_system_cmd {command} {
    

    set status 0
    set errorList {}

    lassign [chan pipe] chStdout chStdin
    lassign [chan pipe] chStderrOut chStderrIn

    try {
        exec >@$chStdin 2>@$chStderrIn {*}"$command"
    } trap NONE errOut {
        # $errOut now holds the message that was written to stderr
        # and everything written to stdout!
    } trap CHILDKILLED {- opts} {
        lassign [dict get $opts -errorcode] pid sigName msg
        # process $pid was killed by signal $sigName; message is $msg
        set status 1
        lappend errorList [list $pid $sigName $msg]
    } trap CHILDSTATUS {- opts} {
        lassign [dict get $opts -errorcode] pid code
        # process $pid exited with non-zero exit code $code
        set status $code
        lappend errorList [list $pid $code]
    } trap CHILDSUSP {- opts} {
        lassign [dict get $opts -errorcode] pid sigName msg
        # process $pid was suspended by signal $sigName; message is $msg
        set status 1
        lappend errorList [list $pid $sigName $msg]
    } trap POSIX {- opts} {
        lassign [dict get $opts -errorcode] errName msg
        # Some kind of kernel failure; details in $errName and $msg
        set status 1
        lappend errorList [list $errName $msg]
    }

    # Get the stdout/stderr from their respective channels and close the
    # channels.
    close $chStdin
    close $chStderrIn
    set stdout [read -nonewline $chStdout]
    set stderr [read -nonewline $chStderrOut]
    close $chStdout
    close $chStderrOut
    # Return a list with the specified order to keep the output format
    # consistant under all circumstances.
    if {![$errorList == "{}"]} {
        return [list $stdout $stderr $status]
    } else {
        eprint "$errorList"
        return [list $stdout $stderr $status $errorList]
    }
}

In [ ]:
%%tcl

exec ls -l

TclError: couldn't execute "ls": no such file or directory

In [19]:
%%tcl
# 09/27

namespace eval options {

    proc options {args} {
        while {[llength $args] > 1} {
            if {[string match [lindex $args 0]* -arrayvariable]} {
                set args [lassign $args _ value]
                set name -arrayvariable
                set options($name) $value
            } else {
                error "Unknown option \"[lindex $args 0]\": must be one of -arrayvariable"
            }
        }
        if {[info exists options(-arrayvariable)]} {
            set upset [format {apply {{name value} {
                    uplevel 2 [list set %s(-$name) $value]
            }}} $value]
        } else {
            set upset [format {apply {{name value} {
                    uplevel 2 [list set $name $value]
            }}}]
        }
        # parse optspec
        set spec [lindex $args 0]
        foreach optspec $spec {
            set name [lindex $optspec 0]
            switch [llength $optspec] {
                1 {
                    dict set opts $name type 0 ;# flag
                    {*}$upset [string range $name 1 end] 0
                    #dict set opts $name value 0
                } 
                2 {
                    dict set opts $name type 1 ;# arbitrary value
                    dict set opts $name default [lindex $optspec 1]
                    {*}$upset [string range $name 1 end] [lindex $optspec 1]
                    #dict set opts $name value [lindex $optspec 1]
                }
                default {
                    dict set opts $name type 2 ;# choice
                    dict set opts $name default [lindex $optspec 1]
                    dict set opts $name values [lrange $optspec 1 end]
                    {*}$upset [string range $name 1 end] [lindex $optspec 1]
                }
            }
        }
        # get caller's args
        upvar 1 args argv
        for {set i 0} {$i<[llength $argv]} {} {
            set arg [lindex $argv $i]
            if {![string match -* $arg]} {
                break
            }
            incr i
            if {$arg eq "--"} {
                break
            }
            set candidates [dict filter $opts key $arg*]
            switch [dict size $candidates] {
                0 {
                    return -code error -level 2 "Unknown option $arg: must be one of [dict keys $opts]"
                }
                1 {
                    dict for {name spec} $candidates {break}
                    set name [string range $name 1 end]
                    dict with spec {} ;# look out
                    if {$type==0} {
                        set value 1
                    } else {
                        if {[llength $argv]<($i+1)} {
                            return -code error -level 2 "Option $name requires a value"
                        }
                        set value [lindex $argv $i]
                        if {$type==2} {
                            set is [lsearch -all -glob $values $value*]
                            switch [llength $is] {
                                1 {
                                    set value [lindex $values $is]
                                }
                                0 {
                                    return -code error -level 2 "Bad $name \"$value\": must be one of $values"
                                }
                                default {
                                    return -code error -level 2 "Ambiguous $name \"$value\": could be any of [lmap i $is {lindex $values $i}]"
                                }
                            }
                        }
                        incr i
                    }
                    {*}$upset $name $value
                }
                default {
                    return -code error -level 2 "Ambiguous option $arg: maybe one of [dict keys $candidates]"
                }
            }
        }
        set argv [lrange $argv $i end]
    }

    proc formatArgspec {argspec} {
        foreach arg $argspec {
            if {[llength $arg]>1} {
                lappend res "?[lindex $arg 0]?"
            } elseif {$arg eq "args"} {
                lappend res "?args ...?"
            } else {
                lappend res $arg
            }
        }
        return [join $res " "]
    }

    proc arguments {argspec} {
        upvar 1 args argv
        for {set i 0} {$i<[llength $argv]} {incr i} {
            if {$i >= [llength $argspec]} {
                return -code error -level 2 "wrong # args: should be \"[lindex [info level -1] 0] [formatArgspec $argspec]\""
            }
            set name [lindex $argspec $i 0]
            if {$name eq "args"} {
                uplevel 1 [list set args [lrange $argv $i end]]
                return
            }
            set value [lindex $argv $i]
#            set test [lindex $argspec $i 2]
#            if {$test != ""} {
#                set valid [uplevel 1 $test $value]
#                if {!$value} {
#                    return -code error -level 2 "Invalid $name \"$value\", must be $test"
#                }
#            }
            uplevel 1 [list set $name $value]
        }
        # defaults:
        for {} {$i < [llength $argspec]} {incr i} {
            set as [lindex $argspec $i]
            if {[llength $as]==1} {
                if {$as ne "args"} {
                    return -code error -level 2 "wrong # args: should be \"[lindex [info level -1] 0] [formatArgspec $argspec]\""
                }
                upvar 1 args args
                set args [lrange $argv $i end]
                return
            }
            lassign $as name value
#            set test [lindex $argspec $i 2]
#            if {$test != ""} {
#                set valid [uplevel 1 $test $value]
#                if {!$value} {
#                    return -code error -level 2 "Invalid $name \"$value\", must be $test"
#                }
#            }
            uplevel 1 [list set $name $value]
        }
    }

    namespace export options arguments
}
namespace import options::*

proc what {args} {
    options {{-loud} {-colour red green blue black} {-count 5}}
    arguments {this {that {}} args}
    foreach name [info locals] {
        puts "$name = [set $name]"
    }
}

what
what -co la

TclError: wrong # args: should be "what this ?that? ?args ...?"

In [20]:
%%tcl
what -co la

TclError: Ambiguous option -co: maybe one of -colour -count

In [21]:
%%tcl 
what -loud

TclError: wrong # args: should be "what this ?that? ?args ...?"